In [ ]:
import csv
!pip install datasets
from datasets import load_dataset
dataset = load_dataset(
    'amazon_us_reviews', 'Mobile_Electronics_v1_00')

In [3]:
import pandas as pd
data = dataset['train'][:]
df = pd.DataFrame(data, columns=data.keys())
training_data = df.sample(frac=0.8, random_state=25)
testing_data = df.drop(training_data.index)
print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 83980
No. of testing examples: 20995


In [5]:
import csv
non_three = open("./annotated_reviews.csv", 'w')
non_three_w = csv.writer(non_three)
non_three_w.writerow(["review", "label"])

three = open("./three_star_reviews.csv", 'w')
three_w = csv.writer(three)
three_w.writerow(["review", "label"])

14

In [ ]:
for row in training_data.iterrows():
    print(row)
    stars = row[1].star_rating
    review = row[1].review_body
    if stars>3:
        non_three_w.writerow([review, 1])
    elif stars<3:
        non_three_w.writerow([review, 2])
    elif stars==3:
        three_w.writerow([review, 3])


In [16]:
from nltk.tokenize import NLTKWordTokenizer
import pandas

annotated_reviews = open("./annotated_reviews.csv", 'r')
annotated_reviews_csv  = pandas.read_csv(annotated_reviews, delimiter = ",", usecols=["review", "label"])
tknzr = NLTKWordTokenizer()
bowListTrain = []
for row in annotated_reviews_csv.iterrows():
    bow = {}
    tokens = tknzr.tokenize(row[1].review.lower())
    for token in tokens:
        if bow.get(token)!=None:
            bow[token] +=1
        else:
            bow[token] = 1
    bowListTrain.append(bow)

In [14]:
three_stars = open("./three_star_reviews.csv", 'r')
three_stars_csv = pandas.read_csv(three_stars, delimiter = ",", usecols=["review", "label"])
bowListTest = []
for row in three_stars_csv.iterrows():
    bow = {}
    tokens = tknzr.tokenize(row[1].review.lower())
    for token in tokens:
        if bow.get(token)!=None:
            bow[token] +=1
        else:
            bow[token] = 1
    bowListTest.append(bow)

In [55]:
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction import DictVectorizer
from sklearn import *
v = DictVectorizer()
X_train = v.fit_transform(bowListTrain)
X_predict = v.transform(bowListTest)

print(X_train.shape)
print(X_predict.shape)

y = annotated_reviews_csv["label"]
model = LogisticRegression('l2')
fitted_model = model.fit(X_train, y)
prediction = fitted_model.predict(X_predict)



(76209, 77965)
(7722, 77965)
[1 2 1 ... 1 1 1]


/Users/elinottonson/.conda/envs/490A/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [59]:
three = open("./three_star_reviews.csv", 'w')
three_w = csv.writer(three)
three_w.writerow(["review", "label", "predicted_label"])
count = 0
for row in three_stars_csv.iterrows():
    print(row)
    stars = row[1].label
    review = row[1].review
    if stars==3:
        three_w.writerow([review, 3, prediction[count]])
        count += 1


(0, review    that would have been nice if it had been updat...
label                                                     3
Name: 0, dtype: object)
(1, review    it said it was new when it was NOT it had been...
label                                                     3
Name: 1, dtype: object)
(2, review    iPhone is too big for pouch.
label                                3
Name: 2, dtype: object)
(3, review    Got it for my 2008 ml350.  Works great for str...
label                                                     3
Name: 3, dtype: object)
(4, review    I purchased this for my mom for Christmas 2012...
label                                                     3
Name: 4, dtype: object)
(5, review    What i didn't like is that the panel its self ...
label                                                     3
Name: 5, dtype: object)
(6, review    This is a very handy unit. Uaing a 3.5mm to st...
label                                                     3
Name: 6, dtype: object)
(7, re